This notebook contains the necessary scripts for making the transform model.

In [1]:
'''Starting with the upload of the necessary libraries including 
the smogn (https://github.com/nickkunz/smogn/tree/master):'''
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy
from scipy.stats import linregress
from scipy.special import boxcox1p
import seaborn as sns
import warnings
import smogn
warnings.filterwarnings('ignore')

C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
### either CPU or GPU:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
### Reading the dummy data
data = pd.read_csv('data/all_data_dummy.csv')

In [4]:
### Applying the SMOGN tranformation on the data
target_variables = ['B', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Be',
                    'As', 'Rb', 'Sr', 'Y', 'Mo', 'Cd', 'Sb', 'Cs', 'Ba', 'Pb']
input_features = ['Li', 'Na', 'Ca', 'SO-', 'Cl-', 'Br-', 'Al', 'Si', 'pH', 'F-', 'Fe', 'Mg']
raw_data = data[target_variables+input_features]

In [5]:
def apply_smogn_safe(data, target_col):
    '''
    This function runs the SMOGN algorithm. 
    For details of SMOGN see https://github.com/nickkunz/smogn/tree/master.
    '''
    try:
        smogn_data = smogn.smoter(
            data=data,
            y=target_col
        )
        return smogn_data, True
    except Exception as e:
        print(f"SMOGN failed: {e}\n→ Using original data instead.")
        return data, False
    
def pytorch_NN(raw_data, input_features, target_variables, smogn_target='V', weight_decay=1e-3, patience=100):
    '''
    This function reads the data and based on the interest of the user, different 
    input features and target variables can be selected from the existing measured
    parameters.
    weight_decay is a regularization technique used to prevent overfitting by penalizing 
    large weights in the model. More details at 
    https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
    patience refers to the number of training epochs to wait after the last improvement 
    in the monitored metric (e.g., validation loss, accuracy) before stopping the training process.
    More details at https://pytorch.org/ignite/generated/ignite.handlers.early_stopping.EarlyStopping.html
    '''
    ### Step 1: Extract input and target
    X = raw_data[input_features]
    y = raw_data[target_variables]

    ### Step 2: Combine into a single DataFrame for easier handling
    full_data = pd.concat([X, y], axis=1)

    ### Step 3: Split into train (60%), val (20%), test (20%)
    train_val_data, test_data = train_test_split(full_data, test_size=0.2)
    train_data, val_data = train_test_split(train_val_data, test_size=0.25)  # 0.25 * 0.8 = 0.2
    
    ### Step 4: Apply SMOGN on training data
    train_data = train_data.reset_index(drop=True)
    train_data_smogn, smogn_success = apply_smogn_safe(train_data, smogn_target)
    
    ### Step 5: Split X and y after SMOGN
    X_train = train_data_smogn[input_features]
    X_train['pH'], _ = scipy.stats.boxcox(X_train['pH']-4)
    X_train['SO-'], _ = scipy.stats.yeojohnson(X_train['SO-']-100)
    X_train['Fe'], _ = scipy.stats.yeojohnson(X_train['Fe'])
    X_train = X_train.to_numpy()
    
    X_val = val_data[input_features]
    X_val['pH'], _ = scipy.stats.boxcox(X_val['pH']-4)
    X_val['SO-'], _ = scipy.stats.yeojohnson(X_val['SO-']-100)
    X_val['Fe'], _ = scipy.stats.yeojohnson(X_val['Fe'])
    X_val = X_val.to_numpy()
    
    X_test = test_data[input_features]
    X_test['pH'], _ = scipy.stats.boxcox(X_test['pH']-4)
    X_test['SO-'], _ = scipy.stats.yeojohnson(X_test['SO-']-100)
    X_test['Fe'], _ = scipy.stats.yeojohnson(X_test['Fe'])
    X_test = X_test.to_numpy()
    
    
    y_train = train_data_smogn[target_variables]
    y_train['Cu'], _ = scipy.stats.yeojohnson(y_train['Cu'])
    y_train['V'], _ = scipy.stats.yeojohnson(y_train['V'])
    y_train['Ni'], _ = scipy.stats.boxcox(y_train['Ni']+1.01)
    y_train['Cs'], _ = scipy.stats.yeojohnson(y_train['Cs']-12)
    y_train['Rb'] = np.sqrt(y_train['Rb'])
    y_train['Y'], _ = scipy.stats.yeojohnson(y_train['Y'])
    y_train['B'], _ = scipy.stats.yeojohnson(y_train['B']-29)
    y_train['Pb'], _ = scipy.stats.yeojohnson(y_train['Pb'])
    y_train = y_train.to_numpy()
    
    y_val = val_data[target_variables]
    y_val['Cu'], _ = scipy.stats.yeojohnson(y_val['Cu'])
    y_val['V'], _ = scipy.stats.yeojohnson(y_val['V'])
    y_val['Ni'], _ = scipy.stats.boxcox(y_val['Ni']+1.01)
    y_val['Cs'], _ = scipy.stats.yeojohnson(y_val['Cs']-12)
    y_val['Rb'] = np.sqrt(y_val['Rb'])
    y_val['Y'], _ = scipy.stats.yeojohnson(y_val['Y'])
    y_val['B'], _ = scipy.stats.yeojohnson(y_val['B']-29)
    y_val['Pb'], _ = scipy.stats.yeojohnson(y_val['Pb'])
    y_val = y_val.to_numpy()
    
    y_test = test_data[target_variables]
    y_test['Cu'], _ = scipy.stats.yeojohnson(y_test['Cu'])
    y_test['V'], _ = scipy.stats.yeojohnson(y_test['V'])
    y_test['Ni'], _ = scipy.stats.boxcox(y_test['Ni']+1.01)
    y_test['Cs'], _ = scipy.stats.yeojohnson(y_test['Cs']-12)
    y_test['Rb'] = np.sqrt(y_test['Rb'])
    y_test['Y'], _ = scipy.stats.yeojohnson(y_test['Y'])
    y_test['B'], _ = scipy.stats.yeojohnson(y_test['B']-29)
    y_test['Pb'], _ = scipy.stats.yeojohnson(y_test['Pb'])
    y_test = y_test.to_numpy()

    ### Step 6: Normalize
    scaler_x = RobustScaler()
    scaler_y = RobustScaler()

    X_train_norm = scaler_x.fit_transform(X_train)
    X_val_norm = scaler_x.transform(X_val)
    X_test_norm = scaler_x.transform(X_test)

    y_train_norm = scaler_y.fit_transform(y_train)
    y_val_norm = scaler_y.transform(y_val)
    y_test_norm = scaler_y.transform(y_test)

    ### Step 7: Convert to PyTorch tensors
    XX = torch.from_numpy(X_train_norm.astype(np.float32)).to(device)
    yy = torch.from_numpy(y_train_norm.astype(np.float32)).to(device)
    XX_val = torch.from_numpy(X_val_norm.astype(np.float32)).to(device)
    yy_val = torch.from_numpy(y_val_norm.astype(np.float32)).to(device)

    input_size = XX.shape[1]
    output_size = yy.shape[1]
    
    ### Step 7: Define the model
    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc1 = nn.Linear(input_size, 32)
            self.norm1 = nn.BatchNorm1d(32)
            self.fc2 = nn.Linear(32, 16)
            self.norm2 = nn.BatchNorm1d(16)
            self.fc3 = nn.Linear(16, output_size)

        def forward(self, x):
            x = torch.relu(self.norm1(self.fc1(x)))
            x = torch.relu(self.norm2(self.fc2(x)))
            return self.fc3(x)

    model = NN().to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=True)

    ### Step 8: Training with early stopping
    best_loss = float('inf')
    patience_counter = 0
    num_epochs = 2000
    loss_train = []
    loss_val = []

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(XX)
        loss = criterion(outputs, yy)
        loss.backward()
        optimizer.step()
        loss_train.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = model(XX_val)
            val_loss = criterion(val_outputs, yy_val)
            loss_val.append(val_loss.item())

        scheduler.step(val_loss)

        if val_loss.item() < best_loss:
            best_loss = val_loss.item()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        if (epoch+1) % 100 == 0:
            print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    print("Training complete.")

    ### Step 9: Evaluate on test set
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.from_numpy(X_test_norm.astype(np.float32)).to(device)
        pred_test = model(X_test_tensor).cpu().numpy()
        pred_test_orig = scaler_y.inverse_transform(pred_test)
        y_test_orig = scaler_y.inverse_transform(y_test_norm)

        pred_train = model(XX).cpu().numpy()
        pred_train_orig = scaler_y.inverse_transform(pred_train)
        y_train_orig = scaler_y.inverse_transform(y_train_norm)

    ### Step 10: R² values
    r_values = []
    for i in range(len(target_variables)):
        r = linregress(y_test_orig[:, i], pred_test_orig[:, i]).rvalue
        r_values.append(r)

    return r_values, y_test_orig, pred_test_orig, y_train_orig, pred_train_orig, loss_train, loss_val

In [6]:
target_variables = ['B', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Be',
                    'As', 'Rb', 'Sr', 'Y', 'Mo', 'Cd', 'Sb', 'Cs', 'Ba', 'Pb'] # to be predicted
input_features = ['Li', 'Na', 'Ca', 'SO-', 'Cl-', 'Br-', 'Al', 'Si', 'pH', 'F-', 'Fe', 'Mg']
import time
start = time.time()
results = []
### Making an ensemble of 1000 models
for i in range(1000):
    print(f'Starting Round {i}')
    r_values, test_y, predicted_y, train_y, predicted_y_train, l_tr, l_te = pytorch_NN(raw_data, input_features, target_variables)
    results.append({
        'r_values': r_values,
        'test_y': test_y,
        'predicted_y': predicted_y,
        'train_y': train_y,
        'predicted_y_train': predicted_y_train,
        'l_tr': l_tr,
        'l_te': l_te,
    })
end = time.time()
print(f'{(end - start)/60} mins')

Starting Round 0


r_index: 100%|##########################################################################| 4/4 [00:00<00:00, 131.19it/s]


Epoch 100, Train Loss: 0.1048, Val Loss: 56.4861
Epoch 00122: reducing learning rate of group 0 to 1.0000e-02.
Early stopping at epoch 199
Training complete.
Starting Round 1


synth_matrix: 100%|######################################################################| 8/8 [00:00<00:00, 44.57it/s]


Epoch 100, Train Loss: 0.0891, Val Loss: 16.5094
Early stopping at epoch 197
Training complete.
Starting Round 2


...

Epoch 100, Train Loss: 0.1186, Val Loss: 1.1165
Epoch 200, Train Loss: 0.1051, Val Loss: 1.0481
Early stopping at epoch 296
Training complete.
Starting Round 998


synth_matrix: 100%|######################################################################| 8/8 [00:00<00:00, 33.92it/s]


Epoch 100, Train Loss: 0.0942, Val Loss: 0.6988
Early stopping at epoch 179
Training complete.
Starting Round 999


r_index: 100%|##########################################################################| 7/7 [00:00<00:00, 159.38it/s]


Epoch 100, Train Loss: 0.1308, Val Loss: 273.2325
Epoch 200, Train Loss: 0.1006, Val Loss: 273.1911
Early stopping at epoch 258
Training complete.
12.857269402345022 mins


In [ ]:
l_te = [[r for r in result['l_te']] for result in results]
l_tr = [[r for r in result['l_tr']] for result in results]
fig, ax = plt.subplots(figsize=(4, 4))
for i in l_tr:
    ax.plot(i, c='b', lw=0.5, label='Train')
    ax.set_yscale('log')
for i in l_te:
    ax.plot(i, c='r', lw=0.5, label='Test')
    ax.set_yscale('log')
ax.set_xlabel('Epochs', fontsize=10, family='Arial')
ax.set_ylabel('MAE', fontsize=10, family='Arial')
plt.yticks(fontname = "Arial", fontsize=10)
plt.xticks(fontname = "Arial", fontsize=10)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize='small')
plt.show()

In [ ]:
### Separating the R2 score of each model
r_squared_values = [[r**2 for r in result['r_values']] for result in results]
df = pd.DataFrame(r_squared_values, columns=target_variables)
df_melted = df.melt(var_name='Element', value_name='R_squared')

In [ ]:
### Making a plot to have a view on the R2 score for each predicted element
fig, ax = plt.subplots(1,1, figsize=(8,3))
sns.boxplot(data=df_melted, x='Element', y='R_squared')
plt.xlabel('Elements', fontname = "Arial", fontsize=10)
plt.ylabel('R$^2$', fontname = "Arial", fontsize=10)
plt.xticks(rotation=90, ha='right')
plt.show()